# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f78148698b0>
├── 'a' --> tensor([[-0.1889,  1.1145, -0.0769],
│                   [ 0.6768,  0.3090, -1.1846]])
└── 'x' --> <FastTreeValue 0x7f7814869ca0>
    └── 'c' --> tensor([[ 0.6928, -1.4038,  1.6759, -0.2158],
                        [-1.1650,  0.9554, -1.5304, -0.6807],
                        [-0.2686, -0.6824,  0.6736, -0.9919]])

In [4]:
t.a

tensor([[-0.1889,  1.1145, -0.0769],
        [ 0.6768,  0.3090, -1.1846]])

In [5]:
%timeit t.a

63.9 ns ± 0.0673 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f78148698b0>
├── 'a' --> tensor([[ 0.5283, -2.2388, -0.7592],
│                   [ 0.9483, -0.6588, -1.2057]])
└── 'x' --> <FastTreeValue 0x7f7814869ca0>
    └── 'c' --> tensor([[ 0.6928, -1.4038,  1.6759, -0.2158],
                        [-1.1650,  0.9554, -1.5304, -0.6807],
                        [-0.2686, -0.6824,  0.6736, -0.9919]])

In [7]:
%timeit t.a = new_value

69.1 ns ± 0.0735 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1889,  1.1145, -0.0769],
               [ 0.6768,  0.3090, -1.1846]]),
    x: Batch(
           c: tensor([[ 0.6928, -1.4038,  1.6759, -0.2158],
                      [-1.1650,  0.9554, -1.5304, -0.6807],
                      [-0.2686, -0.6824,  0.6736, -0.9919]]),
       ),
)

In [10]:
b.a

tensor([[-0.1889,  1.1145, -0.0769],
        [ 0.6768,  0.3090, -1.1846]])

In [11]:
%timeit b.a

60.6 ns ± 0.075 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9380, -0.2270,  1.0033],
               [-0.0183,  0.2787, -0.2312]]),
    x: Batch(
           c: tensor([[ 0.6928, -1.4038,  1.6759, -0.2158],
                      [-1.1650,  0.9554, -1.5304, -0.6807],
                      [-0.2686, -0.6824,  0.6736, -0.9919]]),
       ),
)

In [13]:
%timeit b.a = new_value

501 ns ± 0.125 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

826 ns ± 0.899 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 22 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 782 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7770157f10>
├── 'a' --> tensor([[[-0.1889,  1.1145, -0.0769],
│                    [ 0.6768,  0.3090, -1.1846]],
│           
│                   [[-0.1889,  1.1145, -0.0769],
│                    [ 0.6768,  0.3090, -1.1846]],
│           
│                   [[-0.1889,  1.1145, -0.0769],
│                    [ 0.6768,  0.3090, -1.1846]],
│           
│                   [[-0.1889,  1.1145, -0.0769],
│                    [ 0.6768,  0.3090, -1.1846]],
│           
│                   [[-0.1889,  1.1145, -0.0769],
│                    [ 0.6768,  0.3090, -1.1846]],
│           
│                   [[-0.1889,  1.1145, -0.0769],
│                    [ 0.6768,  0.3090, -1.1846]],
│           
│                   [[-0.1889,  1.1145, -0.0769],
│                    [ 0.6768,  0.3090, -1.1846]],
│           
│                   [[-0.1889,  1.1145, -0.0769],
│                    [ 0.6768,  0.3090, -1.1846]]])
└── 'x' --> <FastTreeValue 0x7f7814862790>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 71.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7769534e20>
├── 'a' --> tensor([[-0.1889,  1.1145, -0.0769],
│                   [ 0.6768,  0.3090, -1.1846],
│                   [-0.1889,  1.1145, -0.0769],
│                   [ 0.6768,  0.3090, -1.1846],
│                   [-0.1889,  1.1145, -0.0769],
│                   [ 0.6768,  0.3090, -1.1846],
│                   [-0.1889,  1.1145, -0.0769],
│                   [ 0.6768,  0.3090, -1.1846],
│                   [-0.1889,  1.1145, -0.0769],
│                   [ 0.6768,  0.3090, -1.1846],
│                   [-0.1889,  1.1145, -0.0769],
│                   [ 0.6768,  0.3090, -1.1846],
│                   [-0.1889,  1.1145, -0.0769],
│                   [ 0.6768,  0.3090, -1.1846],
│                   [-0.1889,  1.1145, -0.0769],
│                   [ 0.6768,  0.3090, -1.1846]])
└── 'x' --> <FastTreeValue 0x7f7769534f10>
    └── 'c' --> tensor([[ 0.6928, -1.4038,  1.6759, -0.2158],
                        [-1.1650,  0.9554, -1.5304, -0.6807],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 57.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.4 µs ± 46.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.6928, -1.4038,  1.6759, -0.2158],
                       [-1.1650,  0.9554, -1.5304, -0.6807],
                       [-0.2686, -0.6824,  0.6736, -0.9919]],
              
                      [[ 0.6928, -1.4038,  1.6759, -0.2158],
                       [-1.1650,  0.9554, -1.5304, -0.6807],
                       [-0.2686, -0.6824,  0.6736, -0.9919]],
              
                      [[ 0.6928, -1.4038,  1.6759, -0.2158],
                       [-1.1650,  0.9554, -1.5304, -0.6807],
                       [-0.2686, -0.6824,  0.6736, -0.9919]],
              
                      [[ 0.6928, -1.4038,  1.6759, -0.2158],
                       [-1.1650,  0.9554, -1.5304, -0.6807],
                       [-0.2686, -0.6824,  0.6736, -0.9919]],
              
                      [[ 0.6928, -1.4038,  1.6759, -0.2158],
                       [-1.1650,  0.9554, -1.5304, -0.6807],
                       [-0.2686, -0.6824,  0.6736, -0.9919]],

In [26]:
%timeit Batch.stack(batches)

76.8 µs ± 40.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.6928, -1.4038,  1.6759, -0.2158],
                      [-1.1650,  0.9554, -1.5304, -0.6807],
                      [-0.2686, -0.6824,  0.6736, -0.9919],
                      [ 0.6928, -1.4038,  1.6759, -0.2158],
                      [-1.1650,  0.9554, -1.5304, -0.6807],
                      [-0.2686, -0.6824,  0.6736, -0.9919],
                      [ 0.6928, -1.4038,  1.6759, -0.2158],
                      [-1.1650,  0.9554, -1.5304, -0.6807],
                      [-0.2686, -0.6824,  0.6736, -0.9919],
                      [ 0.6928, -1.4038,  1.6759, -0.2158],
                      [-1.1650,  0.9554, -1.5304, -0.6807],
                      [-0.2686, -0.6824,  0.6736, -0.9919],
                      [ 0.6928, -1.4038,  1.6759, -0.2158],
                      [-1.1650,  0.9554, -1.5304, -0.6807],
                      [-0.2686, -0.6824,  0.6736, -0.9919],
                      [ 0.6928, -1.4038,  1.6759, -0.2158],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 412 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

329 µs ± 2.14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
